In [ ]:
using JuLIP, JuLIP.ASE, JuLIP.Potentials 
using PyPlot

function plot2d(at, ttl)
    X = positions(at) |> mat 
    plot(X[1,:], X[2,:], "b.", markersize=20)
    title(ttl)
    PyPlot.draw()
    PyPlot.pause(0.0001)    
end 

# For now, this notebook is just to implement a temporary
# example for MD, Newtonian dynamics. This functionality
# should be moved into the JuLIP library, using more
# sophisticated integrators. 

In [ ]:
at = bulk("Si", cubic=true) * (4,4,1)
set_pbc!(at, true)
set_calculator!(at, StillingerWeber())
set_constraint!(at, FixedCell(at));

In [ ]:
# this should be done properly, using in particular 
# a proper scaling of the gaussian depending on temperature 
q = position_dofs(at) 
p = randn(length(q)) * 0.1
set_momentum_dofs!(at, p) 
write("md.xyz", at)
# prepare for plotting
pygui(true)
PyPlot.ion()
PyPlot.hold(false)
# create a counter to save only every 10 steps 
cnt = 0 
dt = 0.003
E0 = energy(at, q)
Ep = [energy(at, q)-E0]
Ek = [0.5 * norm(p)^2] 
for n = 1:10_000
    p -= dt * gradient(at, q)    # Euler-A method
    q += dt * p
    cnt += 1
    push!(Ep, energy(at, q) - E0)
    push!(Ek, 0.5 * vecnorm(p)^2)
    if cnt == 40 
        cnt = 0 
        # write current configuration into at
        set_position_dofs!(at, q) 
        set_momentum_dofs!(at, p)
        # write to file
        write("md.xyz", at, :append)
        plot2d(at, "n = $(n), Ep = $(Ep[end]), Ek = $(Ek[end])") 
    end 
end 


In [ ]:
PyPlot.pygui(false)
x = collect(1:length(Ep)) * dt
plot(x, Ep, "b-", x, Ek, "g-", x, Ep+Ek, "r-")
legend(("E_pot", "E_kin", "E_tot"))